In [41]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [42]:
# BEGIN BASIC AUTH MODULE CODE (Comments removed)
import cookielib
import netrc
import urllib2
import re
 
import pydap.lib
from pydap.exceptions import ClientError
 
import logging
 
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)
 
# Set the debug level for urllib2.
debuglevel=1
 
def install_basic_client(uri='', user='', passwd='', use_netrc=True):
    # Create special opener with support for Cookies
    cj = cookielib.CookieJar()
    
    # Create the password manager and load with the credentials using 
    pwMgr = urllib2.HTTPPasswordMgrWithDefaultRealm()
 
    # Get passwords from the .netrc file nless use_netrc is False    
    if use_netrc:
        logins = netrc.netrc()
        accounts = logins.hosts # a dist of hosts and tuples
        for host, info in accounts.iteritems():
            login, account, password = info
            log.debug('Host: %s; login: %s; account: %s; password: %s' % (host, login, account, password))
            pwMgr.add_password(None, host, login, password)
        
    if uri and user and passwd:
        pwMgr.add_password(None, uri, user, passwd)
    
    opener = urllib2.build_opener(urllib2.HTTPBasicAuthHandler(pwMgr),
                                  urllib2.HTTPCookieProcessor(cj))
    
    opener.addheaders = [('User-agent', 'pydap/EL')]
 
    urllib2.install_opener(opener)
 
    def new_request(url):
        if url[-1] is '&': url = url[0:-1]
        log.debug('Opening %s (install_basic_client)' % url)
        r = urllib2.urlopen(url)
        
        resp = r.headers.dict
        resp['status'] = str(r.code)
        data = r.read()
 
        # When an error is returned, we parse the error message from the
        # server and return it in a ``ClientError`` exception.
        if resp.get("content-description") == "dods_error":
            m = re.search('code = (?P<code>\d+);\s*message = "(?P<msg>.*)"',
                    data, re.DOTALL | re.MULTILINE)
            msg = 'Server error %(code)s: "%(msg)s"' % m.groupdict()
            raise ClientError(msg)
 
        return resp, data
 
    from pydap.util import http
    http.request = new_request
    
# END BASIC AUTH MODULE CODE
 
install_basic_client()
from pydap.client import open_url

ImportError: No module named _LWPCookieJar

In [32]:
loc = 'https://disc2.gesdisc.eosdis.nasa.gov:443/opendap/TRMM_RT/TRMM_3B42RT_Daily.7/'
years = range(2011,2012)
months = ["{0:0=2d}".format(month) for month in range(1,13)]
days = ["{0:0=2d}".format(day) for day in range(1,32)]
paths = []
data = []
fails = []
for year in years:
    for month in months:
        for day in days:
            path=loc+str(year)+'/'+month+'/3B42RT_Daily.'+str(year)+month+day+'.7.nc4'
            paths.append(path)
#             data.append(xr.open_dataset(path))
            try: data.append(netCDF4.Dataset(path)['precipitation'])
            except: fails.append(str(year)+month+day)

In [33]:
fails

['20110111',
 '20110112',
 '20110113',
 '20110114',
 '20110115',
 '20110126',
 '20110127',
 '20110128',
 '20110129',
 '20110209',
 '20110210',
 '20110211',
 '20110212',
 '20110213',
 '20110216',
 '20110225',
 '20110228',
 '20110229',
 '20110230',
 '20110231',
 '20110311',
 '20110313',
 '20110318',
 '20110319',
 '20110320',
 '20110327',
 '20110330',
 '20110402',
 '20110403',
 '20110410',
 '20110413',
 '20110414',
 '20110415',
 '20110425',
 '20110427',
 '20110428',
 '20110431',
 '20110505',
 '20110510',
 '20110511',
 '20110512',
 '20110514',
 '20110516',
 '20110525',
 '20110526',
 '20110527',
 '20110530',
 '20110601',
 '20110608',
 '20110609',
 '20110610',
 '20110615',
 '20110620',
 '20110622',
 '20110624',
 '20110625',
 '20110626',
 '20110627',
 '20110628',
 '20110631',
 '20110709',
 '20110710',
 '20110711',
 '20110722',
 '20110723',
 '20110724',
 '20110725',
 '20110726',
 '20110806',
 '20110807',
 '20110808',
 '20110809',
 '20110810',
 '20110811',
 '20110813',
 '20110814',
 '20110815',

In [26]:
import netCDF4

In [37]:
nc = netCDF4.Dataset(paths[-1])

In [38]:
nc.variables.keys()

[u'precipitation_cnt',
 u'uncal_precipitation',
 u'uncal_precipitation_cnt',
 u'precipitation',
 u'randomError_cnt',
 u'randomError',
 u'lat',
 u'lon']

In [39]:
nc['lat'][:]

RuntimeError: NetCDF: I/O failure